### 40 - Regularization and Cross Validation

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.metrics import mean_absolute_error,mean_squared_error, SCORERS
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from joblib import dump,load

In [36]:
#advertising dataset
df = pd.read_csv('Advertising.csv')
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [37]:
#split into features
X = df.drop('sales', axis=1)
y = df['sales']

In [38]:
#create polynomial features
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [39]:
poly_features = polynomial_converter.fit_transform(X)

In [40]:
#check shape
poly_features.shape

(200, 19)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [42]:
#scale data
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [43]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [44]:
#check X train
X_train[0]

array([ 0.49300171, -0.33994238,  1.61586707,  0.28407363, -0.02568776,
        1.49677566, -0.59023161,  0.41659155,  1.6137853 ,  0.08057172,
       -0.05392229,  1.01524393, -0.36986163,  0.52457967,  1.48737034,
       -0.66096022, -0.16360242,  0.54694754,  1.37075536])

In [45]:
help(Ridge)

Help on class Ridge in module sklearn.linear_model._ridge:

class Ridge(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, _BaseRidge)
 |  Ridge(alpha=1.0, *, fit_intercept=True, copy_X=True, max_iter=None, tol=0.0001, solver='auto', positive=False, random_state=None)
 |  
 |  Linear least squares with l2 regularization.
 |  
 |  Minimizes the objective function::
 |  
 |  ||y - Xw||^2_2 + alpha * ||w||^2_2
 |  
 |  This model solves a regression model where the loss function is
 |  the linear least squares function and regularization is given by
 |  the l2-norm. Also known as Ridge Regression or Tikhonov regularization.
 |  This estimator has built-in support for multi-variate regression
 |  (i.e., when y is a 2d-array of shape (n_samples, n_targets)).
 |  
 |  Read more in the :ref:`User Guide <ridge_regression>`.
 |  
 |  Parameters
 |  ----------
 |  alpha : {float, ndarray of shape (n_targets,)}, default=1.0
 |      Constant that multiplies the L2 term, controlling regula

In [46]:
ridge_model = Ridge(alpha=10)

In [47]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [48]:
test_predictions = ridge_model.predict(X_test)

In [49]:
MAE = mean_absolute_error(y_test, test_predictions)
MAE

0.5774404204714166

In [50]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))
RMSE

0.8946386461319648

In [54]:
#start ridgeCV model and perform cross validation 3 times. 
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0), scoring='neg_mean_absolute_error')
ridge_cv_model.fit(X_train, y_train)

RidgeCV(scoring='neg_mean_absolute_error')

In [55]:
#check best alpha
ridge_cv_model.alpha_

0.1

In [56]:
#check different error scorer metrics
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'positive_likelihood_ratio', 'neg_negative_likelihood_ratio', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weig

In [57]:
test_predictions = ridge_cv_model.predict(X_test)

In [58]:
cv_MAE = mean_absolute_error(y_test, test_predictions)
cv_RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [59]:
cv_MAE

0.42737748843375084

In [60]:
cv_RMSE

0.6180719926926787

In [61]:
#view coefficients
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

### LASSO - Least Absolute Shrinkage and Selection Operator

In [62]:
from sklearn.linear_model import LassoCV

In [72]:
lasso_cv_model = LassoCV(eps=0.1, n_alphas=100, cv=5)

In [73]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(cv=5, eps=0.1)

In [74]:
lasso_cv_model.alpha_

0.4943070909225832

In [77]:
test_predictions = lasso_cv_model.predict(X_test)

In [78]:
lasso_cv_MAE = mean_absolute_error(y_test, test_predictions)
lasso_cv_RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [79]:
lasso_cv_MAE

0.6541723161252867

In [80]:
lasso_cv_RMSE

1.1308001022762548

In [81]:
#check coefficients
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

### L1 + L2 Regularization - Elastic Net

In [82]:
from sklearn.linear_model import ElasticNetCV

In [84]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99 , 1],   eps=0.001, n_alphas=100, max_iter=1000000)

In [85]:
elastic_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [88]:
#find best performing ratio
elastic_model.l1_ratio_

1.0

In [89]:
#predict
test_predictions = elastic_model.predict(X_test)

In [91]:
#find errors
elastic_model_mae = mean_absolute_error(y_test, test_predictions)
elastic_model_mae

0.4335034618590078